In [9]:
import pandas as pd
import os
import numpy as np

In [10]:
import os

dirs = [d for d in os.listdir('./') if os.path.isdir(d)]
print(dirs)
data_hot=pd.DataFrame()
for d in dirs:
    data_i=pd.read_csv(f"./{d}/MEDIS{d[1:]}.TXT",encoding="cp932")
    year=int(d[1:5])
    if d[5:7]=="02":
        year-=1
    data_i["year"]=year
    print(d,year)
    data_hot=pd.concat([data_hot,data_i],axis=0)
data_hot=data_hot.reset_index(drop=True)
# data_hot.head()

['h20140331', 'h20150228', 'h20150331', 'h20160229', 'h20160331', 'h20170228', 'h20170331', 'h20180228', 'h20180331', 'h20190228', 'h20190331', 'h20200229', 'h20200331', 'h20210228', 'h20210331', 'h20220228', 'h20220331']
h20140331 2014
h20150228 2014
h20150331 2015
h20160229 2015
h20160331 2016
h20170228 2016
h20170331 2017
h20180228 2017
h20180331 2018
h20190228 2018
h20190331 2019
h20200229 2019
h20200331 2020
h20210228 2020
h20210331 2021
h20220228 2021
h20220331 2022


In [11]:
data_hot.shape
data_hot["ＪＡＮコード"]=data_hot["ＪＡＮコード"].fillna(0).astype(int).astype(str)
data_hot.columns

c:\Users\hakos\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\dtypes\cast.py:1196: RuntimeWarning: invalid value encountered in cast
  return values.astype(dtype, copy=copy)


Index(['基準番号（ＨＯＴコード）', '処方用番号（ＨＯＴ７）', '会社識別用番号', '調剤用番号', '物流用番号', 'ＪＡＮコード',
       '薬価基準収載医薬品コード', '個別医薬品コード', 'レセプト電算処理システムコード（１）', 'レセプト電算処理システムコード（２）',
       '告示名称', '販売名', 'レセプト電算処理システム医薬品名', '規格単位', '包装形態', '包装単位数', '包装単位単位',
       '包装総量数', '包装総量単位', '区分', '製造会社', '販売会社', '更新区分', '更新年月日', 'year'],
      dtype='object')

In [12]:
data_hot["区分"].value_counts()
# data_hot=data_hot[((data_hot["区分"]=="内") |(data_hot["区分"]=="外"))]
data_hot=data_hot[data_hot["区分"].isin(["内","外"])]
data_hot.drop_duplicates(subset=["個別医薬品コード","year"],inplace=True,keep="first")

In [13]:
data_hot.shape

(150422, 25)

In [14]:
data_hot.groupby(["year","レセプト電算処理システムコード（１）"]).size()

year  レセプト電算処理システムコード（１）
2014  610406008.0           1
      610406009.0           1
      610406013.0           1
      610406015.0           1
      610406047.0           1
                           ..
2022  667990005.0           1
      667990006.0           1
      667990007.0           1
      668110001.0           1
      668110002.0           1
Length: 149622, dtype: int64

In [15]:
# pd.options.display.max_rows = None
data_hot.groupby(["year","レセプト電算処理システムコード（２）"]).size()

year  レセプト電算処理システムコード（２）
2014  610406193.0           1
      610407122.0           1
      610412112.0           2
      610412125.0           1
      610412136.0           2
                           ..
2022  667120028.0           1
      667120031.0           1
      667120034.0           1
      667190004.0           2
      667190005.0           1
Length: 9148, dtype: int64

In [16]:
# data_hot[data_hot["レセプト電算処理システムコード（１）"]==610406002]

In [17]:
data_blp=pd.read_csv("../merged/len3_ndb_blp_DN_firm.csv",encoding="utf-8-sig",index_col=0)

C:\Users\hakos\AppData\Local\Temp\ipykernel_13884\284754022.py:1: DtypeWarning: Columns (118) have mixed types. Specify dtype option on import or set low_memory=False.
  data_blp=pd.read_csv("../merged/len3_ndb_blp_DN_firm.csv",encoding="utf-8-sig",index_col=0)


In [18]:
len(set(data_blp["医薬品コード"])),len(set(data_blp["医薬品名"]))

(2743, 2746)

In [19]:
toitsu_code_set=set(data_blp.loc[data_blp["メーカー名"]=="統一","医薬品コード"])
# data_toitsu=data_blp.loc[data_blp["医薬品コード"].isin(toitsu_code_set),:]
data_toitsu=data_blp.loc[data_blp["メーカー名"]=="統一",:]
data_toitsu.shape,data_blp.shape

((3739, 136), (26198, 136))

In [20]:
# data_merged=pd.merge(data_blp,data_hot,left_on=["医薬品コード","year"],right_on=["レセプト電算処理システムコード（１）","year"],how="left")
data_merged=pd.merge(data_toitsu,data_hot,left_on=["医薬品コード","year"],right_on=["レセプト電算処理システムコード（１）","year"],how="left")
data_merged.shape

c:\Users\hakos\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\reshape\merge.py:1214: RuntimeWarning: invalid value encountered in cast
  if not (rk == rk.astype(lk.dtype))[~np.isnan(rk)].all():


(3739, 160)

In [21]:
len(set(data_blp.loc[data_blp["メーカー名"]=="統一","医薬品名"]))

806

In [22]:
data_map=data_merged[~data_merged["製造会社"].isna()].drop_duplicates(subset=["医薬品コード"])
map=data_map["製造会社"].set_axis(data_map["医薬品コード"]) # map: 医薬品コード to 製造会社
data_blp["メーカー名"]=data_blp["医薬品コード"].map(map).fillna(data_blp["メーカー名"])

In [23]:
len(set(data_blp.loc[data_blp["メーカー名"]=="統一","医薬品名"])),len(set(data_blp.loc[data_blp["メーカー名"].isna(),"医薬品名"]))


(54, 0)

In [24]:
len(set(data_blp["メーカー名"]))
data_blp.to_csv("../merged/len3_ndb_blp_DN_firm_filled.csv",encoding="utf-8-sig")

In [25]:
import sys
sys.exit()

SystemExit: 

c:\Users\hakos\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py:3377: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


製造会社がないものは統一名収載そのものか確かめる

In [ ]:
data_merged["製造会社"].isna()
set(data_merged.loc[data_merged["製造会社"].isna(),"医薬品名"])
rest_code=set(data_merged.loc[data_merged["製造会社"].isna(),"医薬品コード"])
pd.options.display.max_rows = None
print(len(rest_code))
# 真の統一名収載
data_merged.loc[data_merged["医薬品コード"].isin(rest_code),["医薬品名","year"]].value_counts()

56


医薬品名                           year
Ｄソルビトール液　７５％                   2021    2
トルナフタート液　２％                    2016    2
塩酸ロペラミド１ｍｇカプセル                 2016    2
                               2017    2
                               2018    2
                               2019    2
塩酸ロペラミド１ｍｇ錠                    2016    2
                               2017    2
ツロブテロール塩酸塩１ｍｇ錠                 2018    2
塩酸ロペラミド１ｍｇ錠                    2019    2
薬用炭                            2015    2
                               2016    2
                               2017    2
                               2018    2
                               2019    2
ジスチグミン臭化物錠　５ｍｇ                 2021    2
                               2020    2
                               2018    2
                               2017    2
                               2016    2
                               2015    2
トルナフタート液　２％                    2015    2
                               2017    2
ジクロフェナクナトリウム５０ｍｇ坐剤   

In [ ]:
# たしかにそう
for code in rest_code:
    if len(data_hot[data_hot["レセプト電算処理システムコード（２）"]==code])<2:
        print(len(data_hot[data_hot["レセプト電算処理システムコード（２）"]==code]),code)

0 662650007
1 622774600


In [ ]:
X=np.array(data_hot["レセプト電算処理システムコード（１）"].isna())
Y=np.array(data_hot["レセプト電算処理システムコード（２）"].isna())
X.sum(),Y.sum(),
rest_code=rest_code-set(data_hot.loc[~Y,"レセプト電算処理システムコード（２）"])
len(rest_code-set(data_hot.loc[~X,"レセプト電算処理システムコード（１）"])),len(rest_code)

(1, 1)

In [ ]:
data_merged.loc[data_merged["医薬品コード"].isin(rest_code),["医薬品名","year"]].value_counts()

医薬品名                year
イオウ・サリチル酸・チアントール軟膏  2015    1
                    2016    1
                    2017    1
                    2018    1
                    2019    1
                    2020    1
                    2021    1
dtype: int64

レセプト電算コード(１)は常にあり、(２)のみのものはない

In [ ]:
(X&~Y).sum(),(~X&Y).sum(),(Y).sum()+(X&~Y).sum()

(0, 111039, 111088)

In [ ]:
# data_hot["レセプト電算処理システムコード（１）"]=data_hot["レセプト電算処理システムコード（１）"].fillna(0).astype(int)
# data_hot["レセプト電算処理システムコード（２）"]=data_hot["レセプト電算処理システムコード（２）"].fillna(0).astype(int)
data_hot.shape

(150422, 25)

In [ ]:
data_merged=data_blp.merge(data_hot,left_on=["メーカー名","医薬品コード"],right_on=["製造会社","レセプト電算処理システムコード（１）"],how="left")

c:\Users\hakos\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\reshape\merge.py:1214: RuntimeWarning: invalid value encountered in cast
  if not (rk == rk.astype(lk.dtype))[~np.isnan(rk)].all():


In [ ]:
hot_set=set(data_hot["レセプト電算処理システムコード（１）"]).union(set(data_hot["レセプト電算処理システムコード（２）"]))
print(len(toitsu_code_set-hot_set))
print(len(set(data_blp["医薬品コード"])-set(data_hot["レセプト電算処理システムコード（１）"])))
print(len(set(data_blp["医薬品コード"])-set(data_hot["レセプト電算処理システムコード（２）"])))


1
55
2687


In [ ]:
data_hot["receipt"]=data_hot["レセプト電算処理システムコード（１）"].fillna(data_hot["レセプト電算処理システムコード（２）"])

In [ ]:
len(toitsu_code_set-set(data_hot["receipt"]))

55

In [ ]:
pd.options.display.max_rows = None
data_blp.loc[~data_blp["医薬品コード"].isin(hot_set),"医薬品名"].value_counts()
data_blp.loc[~data_blp["医薬品コード"].isin(hot_set),"医薬品コード"].value_counts()

662650007    7
Name: 医薬品コード, dtype: int64

In [ ]:
# data_blp[data_blp["薬価基準収載医薬品コード"]=="2659800M1011"]